# PROYECTO FINAL: MINERÍA DE DATOS MUSICAL
## Metodología KDD (Knowledge Discovery in Databases)

Este proyecto sigue el proceso **KDD**, un estándar clásico para descubrir conocimiento útil a partir de grandes volúmenes de datos. El objetivo es transformar datos crudos de Spotify en reglas de negocio accionables.

**Las 5 Etapas del Proceso KDD:**
1.  **Selección:** Identificación y obtención del conjunto de datos objetivo.
2.  **Preprocesamiento:** Limpieza de datos (ruido, valores faltantes, duplicados).
3.  **Transformación:** Reducción de dimensiones, ingeniería de características y normalización.
4.  **Minería de Datos:** Aplicación de algoritmos inteligentes para extraer patrones (Modelado).
5.  **Interpretación/Evaluación:** Análisis de los patrones descubiertos y su validez para el negocio.

---

## ETAPA 1: SELECCIÓN (Selection)

**Objetivo:** Crear un conjunto de datos objetivo sobre el cual realizar el descubrimiento.

Hemos seleccionado un dataset de **Spotify** que contiene metadatos de audio. Nuestro foco es la variable `popularity` (popularidad), y queremos descubrir qué características acústicas (como el ritmo o la energía) la determinan.

**Variables Clave:**
*   `danceability`: 0.0 a 1.0 (Bailabilidad).
*   `energy`: 0.0 a 1.0 (Intensidad).
*   `loudness`: Volumen en dB.
*   `tempo`: Velocidad en BPM.
*   `popularity`: Índice de éxito (0-100).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Carga de datos (Selección del dataset objetivo)
try:
    df = pd.read_csv('spotify_tracks.csv')
    print(" Datos cargados correctamente. Total de canciones:", len(df))
    print("\n--- Vista Previa (Raw Data) ---")
    print(df[['track_name', 'artists', 'popularity', 'danceability', 'energy']].head(5))
except FileNotFoundError:
    print(" Error: Archivo spotify_tracks.csv no encontrado.")

## ETAPA 2: PREPROCESAMIENTO (Preprocessing)

**Objetivo:** Limpiar y depurar los datos para garantizar calidad. "Garbage in, garbage out".

**Tareas realizadas:**
1.  **Eliminación de Duplicados:** Canciones repetidas (mismo nombre y artista) sesgan el modelo.
2.  **Filtrado de Ruido:** Eliminamos canciones con tempos irreales (< 20 BPM o > 250 BPM).
3.  **Manejo de Nulos:** Se descartan filas con información incompleta.

In [ ]:
# 1. Selección de Variables Base
features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

# 2. Limpieza (Cleaning)
df_clean = df[['popularity', 'track_genre', 'track_name', 'artists'] + features].dropna()
df_clean = df_clean.drop_duplicates(subset=['track_name', 'artists'])

# 3. Filtrado de Outliers (Noise Removal)
df_clean = df_clean[(df_clean['tempo'] > 20) & (df_clean['tempo'] < 250)].reset_index(drop=True)

print(f" Preprocesamiento completado. Registros limpios: {df_clean.shape[0]} (Se eliminaron {len(df) - len(df_clean)} registros sucios/duplicados).")

## ETAPA 3: TRANSFORMACIÓN (Transformation)

**Objetivo:** Proyectar los datos en un formato adecuado para la minería.

**Procesos Clave:**
1.  **Ingeniería de Características (Target Engineering):** Convertimos la popularidad continua (0-100) en una clase binaria para clasificación: **Alto Potencial (1)** vs **Bajo Potencial (0)**.
2.  **Normalización (Scaling):** Usamos `StandardScaler` para poner todas las variables en la misma escala (media 0, desviación 1). Esto es crítico para algoritmos basados en distancia como KNN.
3.  **Análisis de Correlación:** Identificamos qué variables tienen poder predictivo.

In [ ]:
# Tarea 1: Definición del Target (Discretización)
df_clean['target'] = (df_clean['popularity'] >= 50).astype(int)

print("Balance de Clases (1=Alto, 0=Bajo):\n", df_clean['target'].value_counts(normalize=True))

# Tarea 2: Escalado de Datos (Transformation)
X = df_clean[features]
y = df_clean['target']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# División Train/Test (Estrategia de validación)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)
print(" Transformación finalizada. Datos normalizados y divididos.")

### 🔎 Visualización de Relaciones (Parte de Transformación)
Antes de la minería, visualizamos las relaciones para entender la estructura de los datos.

**1. Gráfico de Distribución del Target:** Verificamos el balance de clases.
**2. Mapa de Calor (Correlación):** Identificamos redundancias y predictores fuertes.

In [ ]:
# Visualización 1: Distribución
plt.figure(figsize=(6, 4))
sns.countplot(x=df_clean['target'], palette=['#64748b', '#1DB954'])
plt.title('Distribución de Clases (KDD Target)')
plt.show()

# Visualización 2: Matriz de Correlación
plt.figure(figsize=(10, 8))
sns.heatmap(df_clean[features + ['popularity']].corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Mapa de Calor: Relación entre Variables')
plt.show()

## ETAPA 4: MINERÍA DE DATOS (Data Mining)

**Objetivo:** Aplicar algoritmos inteligentes para extraer patrones ocultos.

Seleccionamos 4 técnicas distintas:
1.  **Árboles de Decisión:** Reglas de decisión jerárquicas.
2.  **KNN (K-Nearest Neighbors):** Clasificación basada en analogía (instancias similares).
3.  **Redes Neuronales (MLP):** Aprendizaje de patrones no lineales complejos.
4.  **K-Means (Clustering):** Segmentación no supervisada (descubrimiento de grupos naturales).

In [ ]:
# ALGORITMO 1: Árbol de Decisión
dt = DecisionTreeClassifier(max_depth=10, random_state=42)
dt.fit(X_train, y_train)
acc_dt = accuracy_score(y_test, dt.predict(X_test))
print(f"⛏️ Minería (Árbol): Precisión de {acc_dt:.4f}")

# ALGORITMO 2: KNN (k=20)
knn = KNeighborsClassifier(n_neighbors=20)
knn.fit(X_train, y_train)
acc_knn = accuracy_score(y_test, knn.predict(X_test))
print(f"⛏️ Minería (KNN): Precisión de {acc_knn:.4f}")

# ALGORITMO 3: Red Neuronal (MLP)
mlp = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=300, random_state=42)
mlp.fit(X_train, y_train)
acc_mlp = accuracy_score(y_test, mlp.predict(X_test))
print(f" Minería (Red Neuronal): Precisión de {acc_mlp:.4f}")

# ALGORITMO 4: Clustering (K-Means)
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
df_clean['cluster'] = kmeans.fit_predict(X_scaled)
print(" Minería (Clustering): Segmentación generada.")

### 🧩 Visualización de Patrones (Resultado de Minería)
Visualizamos los clusters encontrados por K-Means para interpretar los segmentos de mercado.

In [ ]:
# Visualización 3: Clusters
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_clean.sample(5000), x='danceability', y='energy', hue='cluster', palette='viridis', alpha=0.6)
plt.title('Patrones Descubiertos: Clusters de Energía vs Bailabilidad')
plt.xlabel('Bailabilidad')
plt.ylabel('Energía')
plt.legend(title='Grupo (Cluster)')
plt.show()

## ETAPA 5: INTERPRETACIÓN Y EVALUACIÓN (Interpretation/Evaluation)

**Objetivo:** Interpretar los patrones descubiertos y consolidar el conocimiento útil.

En esta fase integramos un modelo de ensamble (**Random Forest**) para mejorar la robustez y evaluamos qué conocimiento hemos extraído.

**Actividades:**
1.  **Evaluación Comparativa:** Validar qué modelo generaliza mejor.
2.  **Meta-Análisis:** Usar Random Forest para determinar la importancia de variables.
3.  **Análisis de Errores:** Matriz de Confusión.

In [ ]:
# Refinamiento con Random Forest (Ensamble)
rf = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
acc_rf = accuracy_score(y_test, y_pred_rf)

print(f" Modelo Final (Random Forest): Precisión {acc_rf:.4f}")

# Comparativa Final
results = pd.DataFrame({
    'Modelo': ['Árbol Decisión', 'KNN', 'Red Neuronal', 'Random Forest'],
    'Precisión': [acc_dt, acc_knn, acc_mlp, acc_rf]
}).sort_values(by='Precisión', ascending=False)

print("\n Interpretación de Resultados:")
print(results)

### 🧠 Interpretación de Conocimiento

**1. Matriz de Confusión:** ¿Dónde falla nuestro conocimiento actual?
**2. Importancia de Variables:** ¿Qué factores gobiernan realmente el éxito musical?

In [ ]:
# Visualización 4: Matriz de Confusión
plt.figure(figsize=(6, 5))
cm = confusion_matrix(y_test, y_pred_rf)
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', xticklabels=['Bajo Potencial', 'Alto Potencial'], yticklabels=['Bajo Potencial', 'Alto Potencial'])
plt.title('Evaluación de Errores (Confusion Matrix)')
plt.ylabel('Realidad')
plt.xlabel('Predicción')
plt.show()

# Visualización 5: Importancia de Variables (Feature Importance)
importances = pd.Series(rf.feature_importances_, index=features).sort_values(ascending=False)
plt.figure(figsize=(10, 6))
sns.barplot(x=importances.values, y=importances.index, palette='viridis')
plt.title('Conocimiento Extraído: Factores Críticos de Éxito')
plt.xlabel('Importancia Relativa')
plt.show()

### 🚀 Aplicación del Conocimiento (Deployment)
Finalmente, el conocimiento "descubierto" (el modelo entrenado) se aplica a nuevos datos para resolver el problema de negocio original.

In [ ]:
# Simulación de Aplicación
new_song_features = np.array([[0.8, 0.9, -5.0, 0.05, 0.1, 0.0, 0.2, 0.8, 128]]) 
new_song_scaled = scaler.transform(new_song_features)
probabilidad = rf.predict_proba(new_song_scaled)[0][1]

print(f"APLICACIÓN DE CONOCIMIENTO:")
print(f"Predicción para nueva canción: {'ALTO POTENCIAL' if probabilidad > 0.5 else 'BAJO POTENCIAL'} ({probabilidad*100:.1f}% confianza)")

In [ ]:
# PERSISTENCIA DE MODELOS (Guardar para App)
import joblib
import os

# Crear carpeta de modelos si no existe
if not os.path.exists('models'):
    os.makedirs('models')

# Guardar objetos clave
joblib.dump(scaler, 'models/scaler.joblib')
joblib.dump(rf, 'models/model_rf.joblib')
joblib.dump(knn, 'models/model_knn.joblib')
joblib.dump(kmeans, 'models/kmeans.joblib')
joblib.dump(features, 'models/features_list.joblib')

print(" Modelos guardados exitosamente en la carpeta 'models/'.")
print("Archivos generados: scaler.joblib, model_rf.joblib, model_knn.joblib, kmeans.joblib, features_list.joblib")